In [1]:
#from IPython.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

### 1 데이터 가져오기(학원 정보)

- 분당구전체정보.csv  다운로드 : http://bit.ly/3Yjm75F   
- 나이스 교육정보 개방포털에서 임의열 편집하여 분당구 데이터만 가지고 왔습니다)
- (원본출처 : https://open.neis.go.kr/portal/data/service/selectServicePage.do?page=1&rows=10&sortColumn=&sortDirection=&infId=OPEN15920190423094641415608&infSeq=1)

In [2]:
import pandas as pd 

In [3]:
#한글 윈도우에서는 cp949 인코딩을 사용하기 때문에 열어보기 위해서 인코딩 설정을 cp949로 합니다.
df = pd.read_csv('분당구전체정보.csv', encoding='cp949')

In [4]:
df.head()

,Unnamed: 0,학원교습소명,학원명,개설일자,등록일자,등록상태명,정원합계,일시수용능력인원합계,분야명,교습계열명,교습과정명,도로명우편번호,도로명주소,도로명상세주소,구
0,0,학원,(2관)기찬에듀기찬수학학원,20200820,20200820,개원,240,95,입시.검정 및 보습,보통교과,보습,13555,경기도 성남시 분당구 정자일로 234,/ 305호 (정자동/ 태남프라자),분당구
1,1,학원,(2관)짱솔학원,20151124,20151124,개원,150,114,입시.검정 및 보습,보통교과,보습,13601,경기도 성남시 분당구 불정로 252,/ 401호/402호 (수내동),분당구
2,2,학원,(분당점)더원학원,20220610,20220610,개원,300,143,입시.검정 및 보습,보통교과,보습,13601,경기도 성남시 분당구 불정로 254,/ 4층 (수내동/ 삼평프라자),분당구
3,3,학원,(주)그래디언트학원,20111205,20111205,개원,75,72,입시.검정 및 보습,보통교과,보습,13558,경기도 성남시 분당구 성남대로331번길 3-13,/ 606호 (정자동/ 대명제스트빌딩),분당구
4,4,학원,(주)아이에스이영어원격학원,20120720,20120720,개원,27,100,입시.검정 및 보습,보통교과,보습,13591,경기도 성남시 분당구 황새울로360번길 21,/ 604호 일부 (서현동/ 분당서현신영팰리스타워),분당구


### 2 주소를 위경도로 변환 (Geocoder)

- 파이썬 geopy 패키지 이용 (X)

위경도 기반 클러스터링 한 결과가 이상해서 확인해보니, 잘못되게 변환 되는 것들이 있었음.

- 구글 스프레드 시트 이용하기 (X)

하루에 변환할 수 있는 개수가 제한이 있는듯 함. (1000건 미만)


- 브이월드 사용(O) 

    * 인증키 발급 받아야함 (쉬움)
    * 사용법 https://www.vworld.kr/dev/v4dv_geocoderguide2_s001.do

#### 2_1 주소 1개로 테스트

In [5]:
# https://www.vworld.kr/dev/v4dv_geocoderguide2_s001.do  에 있는 샘플 예제코드로 테스트 

#해당 정보를 받아오기 위해 HTTP 통신을 사용할 수 있는 requests 를 사용. 
# requests 는 작고 단순한 브라우저로 HTTP 통신을 통해 받은 응답값을 반환함


import requests
apiurl = "http://api.vworld.kr/req/address?"
params = {
    "service": "address",
    "request": "getcoord",
    "crs": "epsg:4326",
    "address": "판교로 242",
    "format": "json", #응답결과 포맷 
    "type": "road",
    "key": "BCAAD83E-1AEF-3DC2-BBBB-068C7C5EACC5" # 각자의 인증키 입력
}
response = requests.get(apiurl, params=params)
if response.status_code == 200:
    json_data = response.json() 
    print(json_data)

{'response': {'service': {'name': 'address', 'version': '2.0', 'operation': 'getcoord', 'time': '108(ms)'}, 'status': 'OK', 'input': {'type': 'road', 'address': '판교로 242'}, 'refined': {'text': '경기도 성남시 분당구 판교로 242 (삼평동)', 'structure': {'level0': '대한민국', 'level1': '경기도', 'level2': '성남시 분당구', 'level3': '삼평동', 'level4L': '판교로', 'level4LC': '', 'level4A': '삼평동', 'level4AC': '4113565500', 'level5': '242', 'detail': ''}}, 'result': {'crs': 'EPSG:4326', 'point': {'x': '127.101313354', 'y': '37.402352535'}}}}


In [6]:
#응답이 dictionary 타입인 것을 알 수 있습니다. 
type(json_data)

dict

In [7]:
# 주소 1개에 대한 응답을 보기 쉽게 펼쳐봅니다.
# 파이썬에서 JSON 형태의 데이터를 처리하기 위해서 사용되는 내장 모듈인 json을 import 합니다. 

# 딕셔너리 자료형을 json 파일로 생성할때는 json.dump 를 사용합니다.
# json.dump 함수는 기본적으로 한글을 아스키 문자열로 변경하기 때문에,
# 가독성이 좋게 ensure_ascii=False 을 주어 아스키 형태로 변환하지 않게 합니다.  
import json

print(json.dumps(json_data, indent="\t", ensure_ascii=False) )

{
	"response": {
		"service": {
			"name": "address",
			"version": "2.0",
			"operation": "getcoord",
			"time": "108(ms)"
		},
		"status": "OK",
		"input": {
			"type": "road",
			"address": "판교로 242"
		},
		"refined": {
			"text": "경기도 성남시 분당구 판교로 242 (삼평동)",
			"structure": {
				"level0": "대한민국",
				"level1": "경기도",
				"level2": "성남시 분당구",
				"level3": "삼평동",
				"level4L": "판교로",
				"level4LC": "",
				"level4A": "삼평동",
				"level4AC": "4113565500",
				"level5": "242",
				"detail": ""
			}
		},
		"result": {
			"crs": "EPSG:4326",
			"point": {
				"x": "127.101313354",
				"y": "37.402352535"
			}
		}
	}
}


In [8]:
#x, y 좌표를 얻기 위하여 해당 key 로 접근합니다. 
json_data['response']['result']['point']

{'x': '127.101313354', 'y': '37.402352535'}

In [9]:
# '도로명주소' 열을 일괄적으로 변경해주기 위하여 apply 에 적용할 함수를 작성합니다. 
# 함수 호출 시 에러가 날 수도 있기 때문에 error 에 대한 처리를 -2, -1 값으로 처리해줍니다. 

import requests

def geocoding_vworld(address_in):

    apiurl = "http://api.vworld.kr/req/address?"
    params = {
        "service": "address",
        "request": "getcoord",
        "crs": "epsg:4326",
        "address": address_in,
        "format": "json",
        "type": "road",
        "key": "AAAAAA-DDDD-CCCC-BBBB-XXXXXXXXXXXX"  #각자 발급받은 키 값 입력으로 수정해야합니다.
    }
    response = requests.get(apiurl, params=params)
    if response.status_code == 200:
        json_data = response.json()
        try:
            return json_data['response']['result']['point']
        except:
            return -2
    else:
        return -1

In [10]:
df['위치'] = df['도로명주소'].apply(geocoding_vworld)

In [11]:
df['위치'].head()

0    {'x': '127.106281974', 'y': '37.371212190'}
1    {'x': '127.126641894', 'y': '37.366216199'}
2    {'x': '127.126822062', 'y': '37.366395367'}
3    {'x': '127.107695832', 'y': '37.364393218'}
4    {'x': '127.125705411', 'y': '37.385710025'}
Name: 위치, dtype: object

In [12]:
df.loc[(df['위치'] == -2) | (df['위치'] == -1)]

,Unnamed: 0,학원교습소명,학원명,개설일자,등록일자,등록상태명,정원합계,일시수용능력인원합계,분야명,교습계열명,교습과정명,도로명우편번호,도로명주소,도로명상세주소,구,위치
817,1124,학원,쁘띠뜨망아트스튜디오미술학원,20180112,20180112,개원,32,42,예능(대),예능(중),미술,13477,경기도 성남시 분당구 운중로233번길 17-12,/ 1층일부 (판교동),분당구,-2
1125,1584,학원,아뿌뚜아미술학원,20180420,20180420,개원,15,40,예능(대),예능(중),미술,13477,경기도 성남시 분당구 운중로267번길 14-6,/ 1층 (판교동),분당구,-2
1328,1863,교습소,연당서예교습소,20130514,20130514,개원,18,9,기타(대),기타(중),서예,13477,경기도 성남시 분당구 운중로233번길 8-2,/ 1층일부 (판교동),분당구,-2


In [13]:
df.drop(df[df['위치'] == -2].index , inplace = True)

In [14]:
df['위치'][0]

{'x': '127.106281974', 'y': '37.371212190'}

In [15]:
type(df['위치'][0])

dict

In [16]:
df['위치'][0]['x']

'127.106281974'

In [17]:
def lat_extract(full_str):

    return full_str['y']

In [18]:
def long_extract(full_str):
    
    return full_str['x']

In [19]:
df['lat'] = df['위치'].apply(lat_extract)

In [20]:
df['lon'] = df['위치'].apply(long_extract)

In [21]:
df['lat'].head()

0    37.371212190
1    37.366216199
2    37.366395367
3    37.364393218
4    37.385710025
Name: lat, dtype: object

In [22]:
df['lon'].head()

0    127.106281974
1    127.126641894
2    127.126822062
3    127.107695832
4    127.125705411
Name: lon, dtype: object

In [23]:
df.drop('위치', axis=1, inplace=True)

In [24]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [25]:
df.head()

,학원교습소명,학원명,개설일자,등록일자,등록상태명,정원합계,일시수용능력인원합계,분야명,교습계열명,교습과정명,도로명우편번호,도로명주소,도로명상세주소,구,lat,lon
0,학원,(2관)기찬에듀기찬수학학원,20200820,20200820,개원,240,95,입시.검정 및 보습,보통교과,보습,13555,경기도 성남시 분당구 정자일로 234,/ 305호 (정자동/ 태남프라자),분당구,37.371212190,127.106281974
1,학원,(2관)짱솔학원,20151124,20151124,개원,150,114,입시.검정 및 보습,보통교과,보습,13601,경기도 성남시 분당구 불정로 252,/ 401호/402호 (수내동),분당구,37.366216199,127.126641894
2,학원,(분당점)더원학원,20220610,20220610,개원,300,143,입시.검정 및 보습,보통교과,보습,13601,경기도 성남시 분당구 불정로 254,/ 4층 (수내동/ 삼평프라자),분당구,37.366395367,127.126822062
3,학원,(주)그래디언트학원,20111205,20111205,개원,75,72,입시.검정 및 보습,보통교과,보습,13558,경기도 성남시 분당구 성남대로331번길 3-13,/ 606호 (정자동/ 대명제스트빌딩),분당구,37.364393218,127.107695832
4,학원,(주)아이에스이영어원격학원,20120720,20120720,개원,27,100,입시.검정 및 보습,보통교과,보습,13591,경기도 성남시 분당구 황새울로360번길 21,/ 604호 일부 (서현동/ 분당서현신영팰리스타워),분당구,37.385710025,127.125705411


In [26]:
df.to_csv('분당구데이터.csv', encoding='cp949', index=False) 